In [1]:
%pip install pandas openpyxl pyautogui

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.2 kB 495.5 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 467.9 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ---------------------------------- ----- 51.2/59.7 kB 2.6 MB/s eta 0:00:01
     ---------------------------------------- 59.7/59.7 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build 


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import webbrowser
import pyautogui
import time
from datetime import datetime
import os
import pyperclip
import re
import numpy as np


pyautogui.PAUSE = 1
pyautogui.FAILSAFE = True
pause_time = 1

In [37]:

df = pd.read_excel('C:/Users/keith/Dropbox/Code/data/MassLotteryGames.xlsx')
print(df)

                           Name  \
0           $10,000,000 BONANZA   
1    $50, $100 and $500 BLOWOUT   
2               GAME OF THRONES   
3           $100,000 CASH EXTRA   
4               $10,000 WIN ALL   
..                          ...   
123               Mega Millions   
124              Lucky for Life   
125                   Megabucks   
126                        Keno   
127               Wheel of Luck   

                                                   URL  Price  Top Prize  \
0    https://www.masslottery.com/games/draw-and-ins...   20.0   10000000   
1    https://www.masslottery.com/games/draw-and-ins...   10.0        500   
2    https://www.masslottery.com/games/draw-and-ins...    5.0     500000   
3    https://www.masslottery.com/games/draw-and-ins...    2.0     100000   
4    https://www.masslottery.com/games/draw-and-ins...    1.0      10000   
..                                                 ...    ...        ...   
123  https://www.masslottery.com/games/draw-and

In [46]:
def extract_prize_tiers(text):
    # Find all prize sections
    prize_sections = re.findall(r'(\$[\d,]+(?:\s\([^)]+\))?)\s*\n\s*1 in ([\d,]+(?:\.\d+)?) odds\s*\n\s*([\d,]+) Start\s*\n\s*([\d,]+) Claimed\s*\n\s*([\d,]+) Remaining', text, re.MULTILINE)

    data = []
    for prize, odds, start, claimed, remaining in prize_sections:
        # Remove commas from numeric values
        odds = odds.replace(',', '')
        start = start.replace(',', '')
        claimed = claimed.replace(',', '')
        remaining = remaining.replace(',', '')

        data.append([
            prize.strip(),
            int(start),
            int(claimed),
            int(remaining),
            f"1/{odds}"
        ])

    # initializing substrings
    sub1 = "approximately "
    sub2 = " tickets."
    
    # getting index of substrings
    try:
        idx1 = text.index(sub1)
        idx2 = text.index(sub2)
    except:
        sub1 = "on the sale of "
        idx1 = text.index(sub1)
        idx2 = text.index(sub2)

    
    res = ''
    # getting elements in between
    for idx in range(idx1 + len(sub1), idx2):
        res = res + text[idx]

    res = res.replace(',', '').replace(' ', '')

    data.append([
            '$0',
            int(res),
            int(0),
            int(res),
            f"1/{1}"
    ])

    # Create a DataFrame
    df = pd.DataFrame(data, columns=['Prize', 'Start', 'Claimed', 'Remaining', 'Odds'])
    return df

def calculate_overall_odds(prize_structure):
    total_tickets = prize_structure['Start'].sum()
    winning_tickets = prize_structure['Start'].sum() - prize_structure.loc[prize_structure['Prize'] == '$0', 'Start'].sum()
    return total_tickets / winning_tickets

def analyze_games(games):
    results = []
    for game_name, prize_structure in games.items():
        # Calculate overall odds
        overall_odds = calculate_overall_odds(prize_structure)
        
        # Calculate expected value
        expected_value = sum((row['Remaining'] / row['Start']) * float(re.sub('[^0-9]','',row['Prize'])) 
                             for _, row in prize_structure.iterrows()) / overall_odds
        # expected_value = sum((row['Remaining'] / row['Start']) * float(row['Prize'].replace('$', '').replace(',', '')) 
        #                      for _, row in prize_structure.iterrows()) / overall_odds
        
        # Calculate percentage of prizes remaining
        total_remaining = prize_structure['Remaining'].sum()
        total_start = prize_structure['Start'].sum()
        percent_remaining = (total_remaining / total_start) * 100
        
        results.append({
            'Game': game_name,
            'Overall Odds': overall_odds,
            'Expected Value': expected_value,
            'Percent Remaining': percent_remaining
        })
    
    return pd.DataFrame(results)

# games = {
#     'Game1': pd.DataFrame({
#         'Prize': ['$500,000', '$500', '$100', '$40', '$20', '$10', '$5,000', '$5', '$1,000', '$0'],
#         'Start': [3, 6048, 37926, 60480, 241920, 907200, 126, 907200, 1449, 9072000],
#         'Claimed': [0, 1533, 9670, 15323, 60848, 224544, 27, 219033, 307, 0],
#         'Remaining': [3, 4515, 28256, 45157, 181072, 682656, 99, 688167, 1142, 9072000],
#         'Odds': ['1/3024000', '1/1500', '1/239.2', '1/150', '1/37.5', '1/10', '1/72000', '1/10', '1/6260.87', '1/1']
#     }),
#     # Add more games here...
# }

for index, row in df.iterrows():

    if index < 118:
        continue

    if index == 120:
        break

    webbrowser.open(row['URL'])
    time.sleep(4)
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.hotkey('ctrl', 'c')
    pyautogui.hotkey('ctrl', 'w')
    text = pyperclip.paste()

    # Extract the prize tiers
    prize_tiers = extract_prize_tiers(text)

    # Sort the DataFrame by the numeric value of the prize
    prize_tiers['PrizeValue'] = prize_tiers['Prize'].str.extract('(\d+)').astype(float)
    prize_tiers = prize_tiers.sort_values('PrizeValue', ascending=False).drop('PrizeValue', axis=1)

    
    games[row['Name']] = prize_tiers


ranked_games = analyze_games(games)
ranked_games_sorted = ranked_games.sort_values('Expected Value')

ticket_prices = df
ticket_prices.drop(['URL', 'Logo URL'], axis=1, inplace=True)
ticket_prices.rename(columns={'Name': 'Game'}, inplace=True)

final_results = pd.merge(ranked_games_sorted, ticket_prices, how='left', on='Game')
display(final_results)


,Game,Overall Odds,Expected Value,Percent Remaining,Price,Top Prize
0,WINTER CASH BLOWOUT,9.875740,2.014688e-01,89.938946,10.0,500
1,Cold Cash Blowout,9.859903,3.755714e-01,89.949659,10.0,500
2,$500 Frenzy,4.456304,1.029122e+00,77.851860,10.0,500
3,Lucky 7s,5.762512,2.533279e+00,83.792248,1.0,10000
4,Quick Silver,5.762512,2.648233e+00,83.950648,1.0,10000
...,...,...,...,...,...,...
116,"$10,000,000 LUCKY BUCKS",3.944958,1.700060e+11,85.868352,20.0,10000000
117,"$10,000,000 BONANZA",3.942973,1.711906e+11,97.550735,20.0,10000000
118,4M Brilliant Titanium,4.475900,1.861824e+11,78.000431,10.0,4000000
119,"$10,000,000 Cash Blast",3.930636,2.554294e+11,88.887327,20.0,10000000


In [47]:
final_results.to_excel('ScratchTicketResults.xlsx', sheet_name='sheet1', index=False)

In [45]:
print(index)

118


In [43]:
print(row['URL'])

https://www.masslottery.com/games/draw-and-instants/10x-bonus-2020


In [35]:
ticket_prices = df
ticket_prices.drop(['URL', 'Logo URL'], axis=1, inplace=True)
ticket_prices.rename(columns={'Name': 'Game'}, inplace=True)

final_results = pd.merge(ranked_games_sorted, ticket_prices, how='left', on='Game')
display(final_results)

,Game,Overall Odds,Expected Value,Percent Remaining,Price,Top Prize
0,Cold Cash Blowout,9.859903,3.757042e-01,89.949731,10.0,500
1,The Instant Game,5.556354,1.426615e+01,84.369111,2.0,50000
2,"$50, $100 and $500 BLOWOUT",9.859903,5.088795e+01,97.667851,10.0,500
3,"$1,000,000 Snow Much Money",5.143999,1.533534e+02,84.596905,5.0,1000000
4,JAWS,4.425554,4.834493e+02,80.985407,10.0,1000000
5,$50 In a Flash,5.764399,5.789084e+02,83.881308,1.0,10000
6,"$10,000 MONOPOLY Doubler",5.764814,6.153710e+02,86.926969,1.0,10000
7,Hot 7s,6.796341,8.122123e+02,90.768217,1.0,7000
8,$50 in a Flash,5.764399,1.167977e+03,95.311869,1.0,10000
9,$50 in a Flash,5.764399,1.167977e+03,95.311869,1.0,10000


In [5]:


start_url = df.URL[10]
webbrowser.open(start_url)
time.sleep(4)
pyautogui.hotkey('ctrl', 'a')
pyautogui.hotkey('ctrl', 'c')
pyautogui.hotkey('ctrl', 'w')
text = pyperclip.paste()
# print(text)


Skip to Main Content





Sign In
Register
Home
>
Games
>
Draw and Instants
>
GAME OF THRONES™
MA-450-Game-Of-Thrones game tile
$5
Top Prize

$500,000
Overall Odds

1 in 4.20
How To Play
Game #450

76.51% Est. Payout

Start Date: 8/6/2024

MA-450-Game-Of-Thrones 2x (1)
Match any of YOUR NUMBERS to any of the WINNING NUMBERS, win prize shown. Get a "COIN" symbol, win prize shown automatically. Get a "2X", "5X", "10X" or "20X" symbol and win 2, 5, 10 or 20 TIMES the prize respectively. Get a "CASTLE" symbol, win $500 instantly.

Prizes
Every effort is made to update this information hourly.
GAME OF THRONES™
Prize	
$500,000

1 in 3,024,000 odds

3 Start

0 Claimed

3 Remaining

$5,000

1 in 72,000 odds

126 Start

27 Claimed

99 Remaining

$1,000

1 in 6,260.87 odds

1,449 Start

307 Claimed

1,142 Remaining

$500

1 in 1,500 odds

6,048 Start

1,533 Claimed

4,515 Remaining

$100

1 in 239.2 odds

37,926 Start

9,670 Claimed

28,256 Remaining

$40

1 in 150 odds

60,480 Start

15,323 Cl

In [21]:
import re
import pandas as pd

def extract_prize_tiers(text):
    # Find all prize sections
    prize_sections = re.findall(r'(\$[\d,]+(?:\s\([^)]+\))?)\s*\n\s*1 in ([\d,]+(?:\.\d+)?) odds\s*\n\s*([\d,]+) Start\s*\n\s*([\d,]+) Claimed\s*\n\s*([\d,]+) Remaining', text, re.MULTILINE)

    data = []
    for prize, odds, start, claimed, remaining in prize_sections:
        # Remove commas from numeric values
        odds = odds.replace(',', '')
        start = start.replace(',', '')
        claimed = claimed.replace(',', '')
        remaining = remaining.replace(',', '')

        data.append([
            prize.strip(),
            int(start),
            int(claimed),
            int(remaining),
            f"1/{odds}"
        ])

    # initializing substrings
    sub1 = "approximately "
    sub2 = " tickets."
    
    # getting index of substrings
    idx1 = text.index(sub1)
    idx2 = text.index(sub2)
    
    res = ''
    # getting elements in between
    for idx in range(idx1 + len(sub1), idx2):
        res = res + text[idx]

    res = res.replace(',', '')

    data.append([
            '$0',
            int(res),
            int(0),
            int(res),
            f"1/{1}"
    ])

    # Create a DataFrame
    df = pd.DataFrame(data, columns=['Prize', 'Start', 'Claimed', 'Remaining', 'Odds'])
    return df



# Extract the prize tiers
prize_tiers = extract_prize_tiers(text)

# Sort the DataFrame by the numeric value of the prize
prize_tiers['PrizeValue'] = prize_tiers['Prize'].str.extract('(\d+)').astype(float)
prize_tiers = prize_tiers.sort_values('PrizeValue', ascending=False).drop('PrizeValue', axis=1)

# Display the table
print(prize_tiers.to_string(index=False))

# Print the number of rows found
print(f"\nNumber of prize tiers found: {len(prize_tiers)}")

# Optionally, save to CSV
prize_tiers.to_csv('prize_tiers.csv', index=False)
print("\nData saved to prize_tiers.csv")

# Debug: Print all prize amounts found in the text
all_prizes = re.findall(r'\$[\d,]+(?:\s\([^)]+\))?', text)
print("\nAll prize amounts found in the text:")
print(all_prizes)

# Debug: Print any prize amounts not included in the final table
table_prizes = set(prize_tiers['Prize'].tolist())
missed_prizes = [prize for prize in all_prizes if prize not in table_prizes]
print("\nPrize amounts not included in the table:")
print(missed_prizes)



   Prize   Start  Claimed  Remaining      Odds
$500,000       3        0          3 1/3024000
    $500    6048     1533       4515    1/1500
    $100   37926     9670      28256   1/239.2
     $40   60480    15323      45157     1/150
     $20  241920    60848     181072    1/37.5
     $10  907200   224544     682656      1/10
  $5,000     126       27         99   1/72000
      $5  907200   219033     688167      1/10
  $1,000    1449      307       1142 1/6260.87
      $0 9072000        0    9072000       1/1

Number of prize tiers found: 10

Data saved to prize_tiers.csv

All prize amounts found in the text:
['$5', '$500,000', '$500', '$500,000', '$5,000', '$1,000', '$500', '$100', '$40', '$20', '$10', '$5']

Prize amounts not included in the table:
[]


In [18]:
test_str = text
 
# printing original string
# print("The original string is : " + str(test_str))
 
# initializing substrings
sub1 = "approximately "
sub2 = " tickets."
 
# getting index of substrings
idx1 = test_str.index(sub1)
idx2 = test_str.index(sub2)
 
res = ''
# getting elements in between
for idx in range(idx1 + len(sub1), idx2):
    res = res + test_str[idx]
 
# printing result
print("The extracted string : " + res)

The extracted string : 9,072,000


In [24]:
import pandas as pd
import numpy as np

def calculate_overall_odds(prize_structure):
    total_tickets = prize_structure['Start'].sum()
    winning_tickets = prize_structure['Start'].sum() - prize_structure.loc[prize_structure['Prize'] == '$0', 'Start'].sum()
    return total_tickets / winning_tickets

def analyze_games(games):
    results = []
    for game_name, prize_structure in games.items():
        # Calculate overall odds
        overall_odds = calculate_overall_odds(prize_structure)
        
        # Calculate expected value
        expected_value = sum((row['Remaining'] / row['Start']) * float(row['Prize'].replace('$', '').replace(',', '')) 
                             for _, row in prize_structure.iterrows()) / overall_odds
        
        # Calculate percentage of prizes remaining
        total_remaining = prize_structure['Remaining'].sum()
        total_start = prize_structure['Start'].sum()
        percent_remaining = (total_remaining / total_start) * 100
        
        results.append({
            'Game': game_name,
            'Overall Odds': overall_odds,
            'Expected Value': expected_value,
            'Percent Remaining': percent_remaining
        })
    
    return pd.DataFrame(results)

# Example usage
games = {
    'Game1': pd.DataFrame({
        'Prize': ['$500,000', '$500', '$100', '$40', '$20', '$10', '$5,000', '$5', '$1,000', '$0'],
        'Start': [3, 6048, 37926, 60480, 241920, 907200, 126, 907200, 1449, 9072000],
        'Claimed': [0, 1533, 9670, 15323, 60848, 224544, 27, 219033, 307, 0],
        'Remaining': [3, 4515, 28256, 45157, 181072, 682656, 99, 688167, 1142, 9072000],
        'Odds': ['1/3024000', '1/1500', '1/239.2', '1/150', '1/37.5', '1/10', '1/72000', '1/10', '1/6260.87', '1/1']
    }),
    # Add more games here...
}

ranked_games = analyze_games(games)
ranked_games_sorted = ranked_games.sort_values('Overall Odds')

print(ranked_games_sorted)

# Optionally, save to CSV
ranked_games_sorted.to_csv('ranked_games.csv', index=False)
print("\nRanked games saved to ranked_games.csv")

    Game  Overall Odds  Expected Value  Percent Remaining
0  Game1      5.195432    97243.242898          95.270889

Ranked games saved to ranked_games.csv


In [23]:
prize_structure = games["Game1"]
print(prize_structure['Start'].sum())
print(prize_structure.loc[prize_structure['Prize'] == '$0', 'Start'].sum())

2162352
0


In [ ]:
pyautogui.hotkey('ctrl', 'a')
pyautogui.hotkey('ctrl', 'c')


In [ ]:
pyautogui.press('enter')